This is a notebook to pull down all the information for figure 3 (ncbi hits). The design is to compair between each sample (barcode) instead of each flowcell as they sequenced the same thing. Therefore I just pulled all the ncbi hits for each flowcell for each barcode and merged them together according to the barcode.

In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import itertools
import argparse
import subprocess

In [2]:
# use argparse to do this
#BASEDIR = args.BASEDIR
BASEDIR = '/home/yiheng/test'

# here we define the folder name of the dataframe it created by capturing the folder from the BASDIR
folder_name = os.path.basename(BASEDIR)
#column_name = folder_name.split('_')[-1]

In [3]:
# first check if the analysis folder is there
folder_list = 'analysis  basecalled_data  scripts  tracking  workspace'.split(' ')
for x in range(0,folder_list.count('')):
    folder_list.remove('')
#fix this test
if not set(os.listdir(os.path.abspath(BASEDIR))) == set (folder_list):
    print("Something wrong with basefolder. check it please.")

In [4]:
# get the dataframe there
dataframe = os.path.join(BASEDIR, 'analysis', 'summary_df_%s.tab' % folder_name)
sum_df = pd.read_csv(dataframe, sep='\t')

/home/yiheng/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1,2,3,5,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# get the dataframe there
tax_linage_dataframe = os.path.join(BASEDIR, 'analysis', '%s_nttaxa.tab' % folder_name)
tax_df = pd.read_csv(tax_linage_dataframe, sep='\t')

/home/yiheng/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
# now just have a few more filtering to make the dataframe smaller so we can process faster
#### first is to select everthing that did hit something
sum_df = sum_df[sum_df.pident_nt > 0]
#### second it needs to pass the filtering steps and has blast hit
sum_df = sum_df[(sum_df.sseqid_nt != False) & (sum_df.passes_filtering == True) & (sum_df.pc_survived == True)] 

In [7]:
#### thirdly just add one more column of the similarity rate for the blast for change parameters of filtering
sum_df['similarity_rate'] = sum_df.length_nt/sum_df.read_length_pc*100

In [8]:
sum_df.head()

,read_id,passes_filtering,sequence_length_template,mean_qscore_template,barcode_arrangement,barcode_score,kit,variant,pc_survived,qseqid_nt,sseqid_nt,evalue_nt,length_nt,pident_nt,nident_nt,sacc_nt,staxids_nt,scomnames_nt,read_length_pc,similarity_rate
25,8b7e2a43-32ae-419c-9326-2cb14eaf1535,True,569,9.69,unclassified,36.4,NB,var1,True,8b7e2a43-32ae-419c-9326-2cb14eaf1535,gi|380353348|emb|HE681722.1|,0.000000e+00,511.0,90.02,460.0,HE681722,273371,Candida orthopsilosis,569.0,89.806678
28,cb3af2d3-7fe3-4bac-8565-0ae34344af18,True,982,8.334,barcode06,94.4615,NB,var1,True,cb3af2d3-7fe3-4bac-8565-0ae34344af18,gi|354543245|emb|HE605203.1|,5.000000e-167,501.0,89.22,447.0,HE605203,5480,Candida parapsilosis,904.0,55.420354
29,d1f8a29b-fcc9-4ff8-812d-9b4cedf53116,True,631,9.865,barcode05,86.9231,NB,var1,True,d1f8a29b-fcc9-4ff8-812d-9b4cedf53116,gi|1069598653|ref|XM_018323264.1|,5.000000e-105,383.0,86.42,331.0,XM_018323264,33203,Purpureocillium lilacinum,560.0,68.392857
42,b50226e5-4abc-4da9-929f-b92f0e7c9eac,True,577,7.061,unclassified,53.641,NB,var1,True,b50226e5-4abc-4da9-929f-b92f0e7c9eac,gi|226874808|ref|NG_011878.1|,2.000000e-59,276.0,84.06,232.0,NG_011878,9606,Homo sapiens,508.0,54.330709
46,1336f3a8-cbe2-4471-9ad4-b5e65e9301b8,True,563,10.385,barcode06,78.8718,NB,var1,True,1336f3a8-cbe2-4471-9ad4-b5e65e9301b8,gi|83764382|dbj|AP007150.1|,0.000000e+00,469.0,92.75,435.0,AP007150,510516,Aspergillus oryzae RIB40,474.0,98.945148


In [9]:
#### drop the colums that we do not need
sum_df = sum_df.drop(columns = ['passes_filtering', 'sequence_length_template', 'mean_qscore_template', 
                                'barcode_arrangement', 'barcode_score', 'sseqid_nt', 'kit',
                                'length_nt', 'staxids_nt', 'nident_nt',
                                'variant', 'pc_survived', 'qseqid_nt', 'sacc_nt', 'scomnames_nt'])

In [10]:
final_df = pd.merge(sum_df, tax_df, how='outer',left_on= 'read_id', right_on='read_id')

In [11]:
final_df.staxids_nt.fillna(False, inplace=True)
final_df = final_df[final_df.staxids_nt != False]
final_df = final_df.reset_index(drop=True)

In [12]:
clinical_df = final_df[final_df.barcode_arrangement == 'barcode07']
#clinical_df.head()

In [13]:
iterables = [ [87],
              [91,92,93,94,95,96,97,98] ]
if not os.path.exists(os.path.join(BASEDIR, 'analysis','blast_pitent_similarity')):
    os.mkdir(os.path.join(BASEDIR, 'analysis','blast_pitent_similarity'))
    
else:
    pass

for t in itertools.product(*iterables):
    clinical_df_fn = os.path.join(BASEDIR, 'analysis','blast_pitent_similarity', 'blast_pitent_similarity_%s_%s.txt' % (t[0], t[1]))
    clinical_df[(clinical_df.pident_nt > t[0]) & (clinical_df.similarity_rate > t[1])].to_csv(clinical_df_fn, sep='\t', index=None)

In [15]:
clinical_df_fn = os.path.join(BASEDIR, 'analysis','blast_pitent_similarity', 'blast_pitent_similarity_89_94.txt')
clinical_df[(clinical_df.pident_nt > 90) & (clinical_df.similarity_rate > 94)].to_csv(clinical_df_fn, sep='\t', index=None)

In [25]:
aa = clinical_df[(clinical_df.pident_nt > 50) & (clinical_df.similarity_rate > 50)]
aa_candida = aa[aa.genus == 'Mus']

In [26]:
aa_candida

,read_id,evalue_nt,pident_nt,read_length_pc,similarity_rate,barcode_arrangement,staxids_nt,superkingdom,phylum,class,order,family,genus,species
20722,643f19a6-aba6-4f17-a7fe-d89708c713e3,2.000000e-165,92.15,708.0,61.158192,barcode07,10090,Eukaryota,Chordata,Mammalia,Rodentia,Muridae,Mus,Mus musculus
37739,3bae5542-5539-438c-9e77-1bf945409df5,5.000000e-142,85.58,899.0,59.399333,barcode07,10090,Eukaryota,Chordata,Mammalia,Rodentia,Muridae,Mus,Mus musculus
62308,a2d87ac3-0513-4bf0-bc68-62616abacc84,6.000000e-159,91.45,593.0,73.018550,barcode07,10090,Eukaryota,Chordata,Mammalia,Rodentia,Muridae,Mus,Mus musculus
64000,d5210def-a1ae-4723-b71b-2935d14f7112,2.000000e-73,85.81,355.0,81.408451,barcode07,10090,Eukaryota,Chordata,Mammalia,Rodentia,Muridae,Mus,Mus musculus
119212,258314a8-ad55-46de-b4c9-99e6b55fef44,2.000000e-54,81.76,551.0,55.716878,barcode07,10090,Eukaryota,Chordata,Mammalia,Rodentia,Muridae,Mus,Mus musculus
158603,d067372b-6210-4947-a338-040d384452fe,2.000000e-61,88.94,339.0,61.356932,barcode07,10090,Eukaryota,Chordata,Mammalia,Rodentia,Muridae,Mus,Mus musculus
161797,6c0a64d2-49c6-481a-98d5-2f6d5314c418,4.000000e-48,91.45,261.0,58.237548,barcode07,10090,Eukaryota,Chordata,Mammalia,Rodentia,Muridae,Mus,Mus musculus
222862,28c37a4a-cb82-4dd2-ba8e-ac64c3293777,2.000000e-62,82.90,372.0,83.333333,barcode07,10090,Eukaryota,Chordata,Mammalia,Rodentia,Muridae,Mus,Mus musculus
234294,8219e832-596a-4b13-b712-92d4f33198b7,3.000000e-101,88.89,370.0,87.567568,barcode07,10090,Eukaryota,Chordata,Mammalia,Rodentia,Muridae,Mus,Mus musculus
249769,dbafa430-0d54-472a-be23-6501e2daf5e0,4.000000e-62,87.18,230.0,101.739130,barcode07,10090,Eukaryota,Chordata,Mammalia,Rodentia,Muridae,Mus,Mus musculus


In [14]:
# generate all the species count files. pretty slow. maybe pool all the slow process to run as a script.
blast_pitent_similarity_path = os.path.join(BASEDIR, 'analysis','blast_pitent_similarity')
species_count_path = os.path.join(blast_pitent_similarity_path, 'species_count')
if not os.path.exists(species_count_path):
    os.mkdir(species_count_path)
else:
    pass


In [ ]:
for file in os.listdir(blast_pitent_similarity_path):
    file_name = os.path.splitext(file)
    file_path = os.path.join(blast_pitent_similarity_path, '%s') % file
    similarity_rate = file_name[0].split('_')[-2]
    pattern_length = file_name[0].split('_')[-1]
    cut_cmd = 'cut -f 14 %s | sort | uniq -c | sort -nr > %s/species_count_%s_%s.txt' % (file_path, species_count_path, similarity_rate, pattern_length)
    cut_cmd_stderr = subprocess.check_output(cut_cmd, shell=True, stderr=subprocess.STDOUT)
    print('species_count_files done.')

In [15]:
counts_70_70 = pd.read_csv(os.path.join(species_count_path, 'species_count_70_70.txt'), header=None)
for items in counts_70_70[0]:
    counts_70_70['number_of_hit']=items.split(' ')[-3]
    print(items.split(' ')[-2])
   # counts_70_70['species']=items.split(' ')[-1]

Homo
Pan
Pseudomonas
Streptococcus
Dracunculus
Gorilla
Candida
Protopolystoma
Pongo
Candida
Pongo
Macaca
Streptococcus
Haemonchus
Rothia
Aspergillus
Spirometra
Pan
Macaca
Streptococcus
[Candida]
Yarrowia
Purpureocillium
Mus
Cryptococcus
Streptococcus
Kluyveromyces
gammaherpesvirus
uncultured
Scedosporium
alphaherpesvirus
Candida
Veillonella
Nomascus
entry
vector
Enterobius
Onchocerca
Streptococcus
Aspergillus
synthetic
Guanarito
Streptococcus
Rhinopithecus
Papio
Meyerozyma
Cercocebus
Wuchereria
uncultured
Streptococcus
Streptococcus
Rhinopithecus
Rattus
Pseudomonas
Prevotella
Pichia
Ovis
Manis
retrovirus
gattii
Chlorocebus
Chlorocebus
Castor
Bos
Aotus
Actinomyces
Wickerhamomyces
Veillonella
uncultured
Triticum
Trichoderma
Tolypocladium
taxon
Sus
sp.
sp.
sp.
Streptococcus
1
sp.
Ralstonia
Oryctolagus
Monodelphis
Miniopterus
Macaca
Hylobates
Gossypium
Eptesicus
Delphinapterus
Cyberlindnera
Cricetulus
Colobus
Clavispora
Cebus
[Candida]
[Candida]
Betapapillomavirus
Aspergillus


In [57]:
items.split(' ')[0]

''

In [ ]:

for counts_file in os.listdir(species_count_path):